# WEB SCRAPING-ASSIGNMENT3

# 1. Write a python program which searches all the product under a particular product from www.amazon.in. 
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search 
for guitars.

Importing important libraries

In [13]:
import pandas as pd
import selenium
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import requests
import re
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings('ignore')

In [14]:
driver=webdriver.Chrome(r"C:\Chrome driver\Chromedriver.exe")

In [15]:
driver.get('https://www.amazon.in/')

In [16]:
# Entering the product that we want to search
product_name = input('Enter the product that you want to search : ')

Enter the product that you want to search : guitar


In [17]:
# add user's data to keyword field 
search_product=driver.find_element(By.XPATH,'//input[@id="twotabsearchtextbox"]')
search_product.send_keys(product_name)

# click on search button

search=driver.find_element(By.XPATH,'//input[@id="nav-search-submit-button"]')
search.click()

# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [18]:
# Fetching URLs to open the pages
product_urls = []    # Empty list
start=0
end=3
for page in range(start,end):   # for loop to scrape 3 pages
    url = driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    for i in url:
        product_urls.append(i.get_attribute("href"))
    nxt_btn=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    nxt_btn.click()
    time.sleep(2)

In [19]:
len(product_urls)

184

In [20]:
# making empty list and fetching required data
brand_name = []
product_name = []
ratings = []
num_ratings = []
prices = []
exchange = []
exp_delivery = []
availability = []
other_details = []

In [21]:
for url in product_urls:
    driver.get(url)
    time.sleep(2)
    
    
    #fetching brand name 
    try:
        brand = driver.find_element(By.XPATH,"//a[@id='bylineInfo']")
        brand_name.append(brand.text)
    except NoSuchElementException:
        brand_name.append('-')
    
    
    # fetching Name of the Product
    try:
        product = driver.find_element(By.XPATH,"//span[@id='productTitle']")
        product_name.append(product.text)
    except NoSuchElementException:
        product_name.append('-')
        
        

     #fetching ratings
    try:
        rating = driver.find_element(By.XPATH,"//span[@class='a-size-base a-nowrap']//span")
        ratings.append(rating.text)
    except NoSuchElementException:
        ratings.append('-')
        
 
    #fetching  no of ratings
    try:
        num_rating = driver.find_element(By.XPATH,"//span[@id='acrCustomerReviewText']")
        num_ratings.append(num_rating.text)
    except NoSuchElementException:
        num_ratings.append('-')
        

    #fetching price of the product
    try:
        price = driver.find_element(By.XPATH,'//span[@class="a-offscreen"]')
        prices.append(price.text)
    except NoSuchElementException:
        prices.append('-')
        
        
    #fetching return/exchange
    try:
        exch = driver.find_element(By.XPATH,"//span[@class='a-declarative']/div/a")
        exchange.append(exch.text)
    except NoSuchElementException:
        exchange.append('-')
        

    #fetching expected delivery
    try:
        delivery = driver.find_element(By.XPATH,"//div[@class='a-section a-spacing-mini']/b")
        exp_delivery.append(delivery.text)
    except NoSuchElementException:
        exp_delivery.append('-')
        

    #fetching availability information
    try:
        avail = driver.find_element(By.XPATH,"//span[@class='a-size-medium a-color-success']")
        availability.append(avail.text)
    except NoSuchElementException:
        availability.append('-')
        
    #other details
    try:
        oth_det = driver.find_element(By.XPATH,"//ul[@class='a-unordered-list a-vertical a-spacing-mini']")
        other_details.append(oth_det.text)
    except NoSuchElementException:
        other_details.append('-')

In [22]:
print(len(brand_name),
len(product_name),
len(ratings),
len(num_ratings),
len(prices),
len(exchange),
len(exp_delivery),
len(availability),
len(other_details))

184 184 184 184 184 184 184 184 184


In [23]:
# Creating the DataFrame for the scraped data

guitar = pd.DataFrame({})
guitar['Brand Name'] = brand_name
guitar['Name of the Product'] = product_name
guitar['Rating'] = ratings
guitar['No. of Ratings'] = num_ratings
guitar['Price'] = prices
guitar['Return/Exchange'] = exchange
guitar['Expected Delivery'] = exp_delivery
guitar['Availability'] = availability
guitar['Other Details'] = other_details
guitar['Product URL'] = product_urls
guitar

,Brand Name,Name of the Product,Rating,No. of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,Product URL
0,Visit the Kadence Store,Kadence Slowhand Premium Jumbo Semi Acoustic G...,4 out of 5,513 ratings,,7 Days Replacement,-,In stock.,ACOUSTIC-ELECTRIC GUITAR: Kadence Slowhand sem...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Visit the Fender Store,Fender. Fender Acoustic Guitar FA115 Dreadnoug...,5 out of 5,2 ratings,,7 Days Replacement,-,Only 1 left in stock.,Body Top - Laminated Spruce Body Back - Lamina...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Visit the blueberry Store,"Blueberry 38C, 38"" Acoustic Guitar Kit with Ba...",3.7 out of 5,20 ratings,,7 Days Replacement,-,Only 2 left in stock.,"Material: Top: linden wood, Side & Back: - lin...",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Brand: Hontral,Hontral S4 String Decor Guitar Children's Musi...,-,-,,7 Days Replacement,-,In stock.,4-string Wooden Guitar Toy with Adjustable Tun...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",4 out of 5,"17,590 ratings",,7 Days Replacement,-,In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
179,Visit the JUAREZ Store,"Juarez JRZ-ST01 6-String Electric Guitar, Righ...",4 out of 5,268 ratings,,7 Days Replacement,-,,Number of Frets: 18\nGreat looks with an innov...,https://www.amazon.in/Juarez-JRZ-ST01-6-String...
180,Visit the MUSTANG Store,Pack of 2 Universal Guitar Stand by Hola! Musi...,3.3 out of 5,26 ratings,,7 Days Replacement,-,In stock.,Music Professional Tripod Universal Guitar Sta...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
181,Visit the SG Musical Store,SG Musical - Guitar Bridge Pins 6 pc,-,-,,7 Days Replacement,-,In stock.,High quality bridge pins\nFit for acoustic gui...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
182,Visit the DEVICE OF URBAN INFOTECH Store,DEVICE OF URBAN INFOTECH Tuning Peg Silver for...,3.4 out of 5,15 ratings,,7 Days Replacement,-,In stock.,"Package Includes: 6 X Machine heads(3L + 3R),1...",https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


In [24]:
#saving the data in csv
guitar.to_csv("Guitar.csv")

# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [36]:
driver=webdriver.Chrome(r"C:\Chrome driver\Chromedriver.exe")

In [37]:
# Getting the webpage of mentioned url 
url = "https://images.google.com/"
#Creating empty list 
urls = []    #empty list
data = []    #empty list
search_item = ["fruits", "cars", "Machine Learning","Guitar","Cakes"]
for item in search_item:
    driver.get(url)  
    time.sleep(5)
    
    # Finding webelement for search_bar
    search_bar = driver.find_element(By.XPATH,"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input") 
    
    # Sending keys to get the keyword for search bar
    search_bar.send_keys(str(item))
    
    # Clicking on search button
    search_button =driver.find_element(By.XPATH,"//button[@class='Tg7LZd']").click()
    
    # Scrolling down the webpage to get some more images
    for _ in range(500):
        driver.execute_script("window.scrollBy(0,100)")
        
        imgs = driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:100]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 300:
        break
    print("Downloading {0} of {1} images" .format(i, 300))
    response = requests.get(urls[i])

    file = open(r"C:\Users\acer\Desktop\Internship Flip Robo\web "+str(i)+".jpg", "wb")

    file.write(response.content)
  

ElementClickInterceptedException: Message: element click intercepted: Element <button jsname="Tg7LZd" class="Tg7LZd" aria-label="Google Search" type="submit" data-ved="0ahUKEwi4u96AqPH6AhWZtGMGHaifBUkQ4dUDCAc">...</button> is not clickable at point (786, 201). Other element would receive the click: <iframe role="presentation" frameborder="0" scrolling="no" name="callout" src="https://ogs.google.com/widget/callout?prid=19028915&amp;pgid=19027681&amp;puid=3e9e72cbc9a83b93&amp;cce=1&amp;dc=1&amp;origin=https%3A%2F%2Fimages.google.com&amp;cn=callout&amp;pid=1&amp;spid=2&amp;hl=en" style="height: 100%; width: 100%;" data-iml="1666354972578"></iframe>
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00CB1ED3+2236115]
	Ordinal0 [0x00C492F1+1807089]
	Ordinal0 [0x00B566FD+812797]
	Ordinal0 [0x00B8BEC7+1031879]
	Ordinal0 [0x00B89E6C+1023596]
	Ordinal0 [0x00B87A5B+1014363]
	Ordinal0 [0x00B866E7+1009383]
	Ordinal0 [0x00B7C416+967702]
	Ordinal0 [0x00BA1A8C+1120908]
	Ordin

# Q4: Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [38]:
driver=webdriver.Chrome(r"C:\Chrome driver\Chromedriver.exe")

In [39]:
driver.get('https://www.flipkart.com/')

In [40]:
# Closing Login popup button
login_X_btn = driver.find_element(By.XPATH,"//div[@class='_2QfC02']//button").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_2QfC02']//button"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00CB1ED3+2236115]
	Ordinal0 [0x00C492F1+1807089]
	Ordinal0 [0x00B566FD+812797]
	Ordinal0 [0x00B855DF+1005023]
	Ordinal0 [0x00B857CB+1005515]
	Ordinal0 [0x00BB7632+1209906]
	Ordinal0 [0x00BA1AD4+1120980]
	Ordinal0 [0x00BB59E2+1202658]
	Ordinal0 [0x00BA18A6+1120422]
	Ordinal0 [0x00B7A73D+960317]
	Ordinal0 [0x00B7B71F+964383]
	GetHandleVerifier [0x00F5E7E2+2743074]
	GetHandleVerifier [0x00F508D4+2685972]
	GetHandleVerifier [0x00D42BAA+532202]
	GetHandleVerifier [0x00D41990+527568]
	Ordinal0 [0x00C5080C+1837068]
	Ordinal0 [0x00C54CD8+1854680]
	Ordinal0 [0x00C54DC5+1854917]
	Ordinal0 [0x00C5ED64+1895780]
	BaseThreadInitThunk [0x767AFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77697BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77697B8E+238]


In [41]:
# Searching for web element
search_bar = driver.find_element(By.XPATH,"//input[@class='_3704LK']")
# Sending keys to search product
search_bar.send_keys("pixel 4A")

In [42]:
# Locating the search button using xpath
search_btn = driver.find_element(By.XPATH,"//button[@class='L0Z3Pu']")
# Clicking on search button
search_btn.click()

In [43]:
# Fetching 1st page URLs of smartphone
page1_url = []   #empty list
urls = driver.find_elements(By.XPATH,"//a[@class='_1fQZEK']")
for i in urls:
    page1_url.append(i.get_attribute('href'))

In [44]:
len(page1_url)

24

In [45]:
Smartphones = {}
Smartphones["Brand"] = []
Smartphones["Phone name"] = []
Smartphones["Colour"] = []
Smartphones["RAM"] = []
Smartphones["Storage(ROM)"] = []
Smartphones["Primary Camera"] = []
Smartphones["Secondary Camera"] = []
Smartphones["Display Size"] = []
Smartphones["Display Resolution"] = []
Smartphones["Processor"] = []
Smartphones["Processor Cores"] = []
Smartphones["Battery Capacity"] = []
Smartphones["Price"] = []
Smartphones["URL"] = []

In [46]:
# scraping data from each url of page 1
for url in page1_url:
    driver.get(url)
    print("Scraping URL = ",url)
    Smartphones['URL'].append(url)
    time.sleep(2)
    
    #clicking on read more button to get more information
    try:
        read_more = driver.find_element(By.XPATH,"//button[@class='_2KpZ6l _1FH0tX']")
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
        
    #scraping brand name of smartphone
    try:
        brand_tags = driver.find_element(By.XPATH,"//span[@class='B_NuCI']")
        Smartphones['Brand'].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand'].append('-')
    
    # scraping name of smartphones
    try:
        name_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][1]/table/tbody/tr[3]/td[2]/ul/li")
        Smartphones['Phone name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Phone name'].append('-')
        
    #scraping colour of smartphone
    try:
        color_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][1]/table/tbody/tr[4]/td[2]/ul/li")
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
        
    # scraping RAM data of smartphone
    try:
        ram_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][4]/table[1]/tbody/tr[2]/td[2]/ul/li")
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
        
    #scraping ROM data of smartphones
    try:
        rom = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][4]/table[1]/tbody/tr[1]/td[2]/ul/li")
        Smartphones['Storage(ROM)'].append(rom.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
    # scraping  Primary camera data of smartphone
    try:
        pri =driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][5]/table[1]/tbody/tr[2]/td[2]/ul/li")
        Smartphones['Primary Camera'].append(pri.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
     
    # scraping secondary camera data of smartphone 
    try:
        sec_cam = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][5]/table[1]/tbody/tr[6]/td[2]/ul/li")
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
     #scraping display size data of smartphone
    try:
        disp = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][2]/div")
        if disp.text != 'Display Features' : raise NoSuchElementException
        disp_size = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][2]/table[1]/tbody/tr[1]/td[2]/ul/li")
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')
        
     #scraping display resolution of smartphone
    try:
        disp = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][2]/div")
        if disp.text != 'Display Features' : raise NoSuchElementException
        disp_reso = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][2]/table[1]/tbody/tr[2]/td[2]/ul/li")
        Smartphones['Display Resolution'].append(disp_reso.text)
    except NoSuchElementException:
        Smartphones['Display Resolution'].append('-')
          
    #scraping processor of smartphone
    try:
        processor = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[2]/td[2]/ul/li")
        Smartphones['Processor'].append(processor.text)
    except NoSuchElementException:
        Smartphones['Processor'].append('-')
        
    # scraping processor core of smartphone
    try:
        core = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[3]/td[1]")
        if core.text != 'Processor Core' :
            core = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[2]/td[1]")
            if core.text != 'Processor Core' :
                raise NoSuchElementException
            else :
                cores = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[2]/td[2]/ul/li")
        else :
            cores = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][3]/table[1]/tbody/tr[3]/td[2]/ul/li")
        Smartphones['Processor Cores'].append(disp_reso.text)
    except NoSuchElementException:
        Smartphones['Processor Cores'].append('-')
        
        
        
    # scraping the battery capacity of smartphone
    try:
        if driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][10]/div").text != "Battery & Power Features" :
            if driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][9]/div").text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][9]/table/tbody/tr/td[1]")
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][9]/table/tbody/tr/td[2]/ul/li")
            elif driver.find_element_by_xpath("//div[@class='_3k-BhJ'][8]/div").text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][8]/table/tbody/tr/td[1]")
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][8]/table/tbody/tr/td[2]/ul/li")
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][10]/table/tbody/tr/td[1]")
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element(By.XPATH,"//div[@class='_3k-BhJ'][10]/table/tbody/tr/td[2]/ul/li")
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
    
    
    # scraping price of smartphone
    try:
        price_tags = driver.find_element(By.XPATH,"//div[@class='_30jeq3 _16Jk6d']")
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
        Smartphones['Price'].append('-')

Scraping URL =  https://www.flipkart.com/google-pixel-4a-just-black-128-gb/p/itm023b9677aa45d?pid=MOBFUSBNAZGY7HQU&lid=LSTMOBFUSBNAZGY7HQUWHTF0C&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=8e1fe8b5-3391-4734-91bb-6f7e86aa0d04.MOBFUSBNAZGY7HQU.SEARCH&ppt=hp&ppn=homepage&ssid=mwnnkjtvls0000001666355263303&qH=9b26a23b2cff510d
Scraping URL =  https://www.flipkart.com/realme-c30-lake-blue-32-gb/p/itmfcf69b2db5059?pid=MOBGF37GTRVMRJVK&lid=LSTMOBGF37GTRVMRJVKRYAJNG&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=8e1fe8b5-3391-4734-91bb-6f7e86aa0d04.MOBGF37GTRVMRJVK.SEARCH&ppt=hp&ppn=homepage&ssid=mwnnkjtvls0000001666355263303&qH=9b26a23b2cff510d
Scraping URL =  https://www.flipkart.com/realme-c30-denim-black-32-gb/p/itmfcf69b2db5059?pid=MOBGF37G8TGKHFWJ&lid=LSTMOBGF37G8TGKHFWJIST95C&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_3&otracker=search&otracker1=search

In [47]:
print(len(Smartphones["Brand"]),
len(Smartphones["Phone name"]),
len(Smartphones["Colour"]),
len(Smartphones["RAM"]),
len(Smartphones["Storage(ROM)"]),
len(Smartphones["Primary Camera"]),
len(Smartphones["Secondary Camera"]),
len(Smartphones["Display Size"]),
len(Smartphones["Display Resolution"]),
len(Smartphones["Processor"]),
len(Smartphones["Processor Cores"]),
len(Smartphones["Battery Capacity"]),
len(Smartphones["Price"]),
len(Smartphones["URL"]))

24 24 24 24 24 24 24 24 24 24 24 24 24 24


In [48]:
# framing the DataFrame

df = pd.DataFrame.from_dict(Smartphones)
df

,Brand,Phone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,URL
0,Google,Pixel 4a,Just Black,6 GB,128 GB,12.2MP Rear Camera,"8MP Front Camera, 1.12 μm Pixel Size, F/2.0 Ap...",14.76 cm (5.81 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 730G,2340 x 1080 Pixels,3140 mAh,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,realme,C30,Lake Blue,2 GB,32 GB,8MP Rear Camera,"5MP Camera Setup: (SmartSens SC501CS-CRMNN00, ...",16.51 cm (6.5 inch),1600 x 720 Pixel,Unisoc T612,1600 x 720 Pixel,5000 mAh,"₹6,299",https://www.flipkart.com/realme-c30-lake-blue-...
2,realme,C30,Denim Black,2 GB,32 GB,8MP Rear Camera,"5MP Camera Setup: (SmartSens SC501CS-CRMNN00, ...",16.51 cm (6.5 inch),1600 x 720 Pixel,Unisoc T612,1600 x 720 Pixel,5000 mAh,"₹6,299",https://www.flipkart.com/realme-c30-denim-blac...
3,realme,C30,Bamboo Green,2 GB,32 GB,8MP Rear Camera,"5MP Camera Setup: (SmartSens SC501CS-CRMNN00, ...",16.51 cm (6.5 inch),1600 x 720 Pixel,Unisoc T612,1600 x 720 Pixel,5000 mAh,"₹6,299",https://www.flipkart.com/realme-c30-bamboo-gre...
4,realme,C30,Bamboo Green,3 GB,32 GB,8MP Rear Camera,"5MP Camera Setup: (SmartSens SC501CS-CRMNN00, ...",16.51 cm (6.5 inch),1600 x 720 Pixel,Unisoc T612,1600 x 720 Pixel,5000 mAh,"₹6,499",https://www.flipkart.com/realme-c30-bamboo-gre...
5,realme,C30,Lake Blue,3 GB,32 GB,8MP Rear Camera,"5MP Camera Setup: (SmartSens SC501CS-CRMNN00, ...",16.51 cm (6.5 inch),1600 x 720 Pixel,Unisoc T612,1600 x 720 Pixel,5000 mAh,"₹6,499",https://www.flipkart.com/realme-c30-lake-blue-...
6,realme,C30,Denim Black,3 GB,32 GB,8MP Rear Camera,"5MP Camera Setup: (SmartSens SC501CS-CRMNN00, ...",16.51 cm (6.5 inch),1600 x 720 Pixel,Unisoc T612,1600 x 720 Pixel,5000 mAh,"₹6,499",https://www.flipkart.com/realme-c30-denim-blac...
7,vivo,T1 44W,Starry Sky,4 GB,128 GB,50MP + 2MP + 2MP,16MP Camera Setup: (f/2.0 Aperture),16.36 cm (6.44 inch),2400 x 1080 Pixel,Qualcomm Snapdragon 680,2400 x 1080 Pixel,5000 mAh,"₹14,499",https://www.flipkart.com/vivo-t1-44w-starry-sk...
8,vivo,T1 44W,Midnight Galaxy,4 GB,128 GB,50MP + 2MP + 2MP,16MP Camera Setup: (f/2.0 Aperture),16.36 cm (6.44 inch),2400 x 1080 Pixel,Qualcomm Snapdragon 680,2400 x 1080 Pixel,5000 mAh,"₹14,499",https://www.flipkart.com/vivo-t1-44w-midnight-...
9,vivo,T1 44W,Starry Sky,6 GB,128 GB,50MP + 2MP + 2MP,16MP Camera Setup: (f/2.0 Aperture),16.36 cm (6.44 inch),2400 x 1080 Pixel,Qualcomm Snapdragon 680,2400 x 1080 Pixel,5000 mAh,"₹15,999",https://www.flipkart.com/vivo-t1-44w-starry-sk...


In [49]:
# saving the data in csv
df.to_csv("smartphones.csv")

# Q5 : Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [50]:
# connecting to the webdriver
driver=webdriver.Chrome(r"C:\Chrome driver\Chromedriver.exe")

In [52]:
driver.get("https://www.google.co.in/maps")

In [53]:
# entering the city name in search bar
City = input('Enter City name that has to be searched : ')
search_bar = driver.find_element(By.XPATH,'/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/form/div[2]/div[3]/div/input[1]')
search_bar.click()
time.sleep(2)

#sending keys to find cities
search_bar.send_keys(City)

#checking for webelement and clicking on search button
search_btn = driver.find_element(By.XPATH,'/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button')
search_btn.click()
time.sleep(2)

try:
    url_str = driver.current_url
    print("URL Extracted: ", url_str)
    latitude_longitude = re.findall(r'@(.*)data',url_str)
    if len(latitude_longitude):
        lat_lng_list = latitude_longitude[0].split(",")
        if len(lat_lng_list)>=2:
            latitude = lat_lng_list[0]
            longitude = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(latitude, longitude))
except Exception as e:
        print("Error: ", str(e))

Enter City name that has to be searched : pune
URL Extracted:  https://www.google.co.in/maps/search/pune/@19.125897,74.7453997,15z


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [54]:
# connecting to the webdriver
driver=webdriver.Chrome(r"C:\Chrome driver\Chromedriver.exe")


In [55]:
driver.get("https://trak.in/")


In [56]:
# getting xpath for funding deals and clicking on the button
fund_button = driver.find_element(By.XPATH,"//li[@id='menu-item-51510']/a").get_attribute('href')
driver.get(fund_button)


In [57]:
#Empty Lists
fund_deals = {}
fund_deals['Date'] = []
fund_deals['Startup Name'] = []
fund_deals['Industry/Vertical'] = []
fund_deals['Sub_Vertical'] = []
fund_deals['Location'] = []
fund_deals['Investor'] = []
fund_deals['Investment Type'] = []
fund_deals['Amount(in USD)'] = []

In [58]:
for i in range(48,51):
    
    # scraping data of data
    date = driver.find_elements(By.XPATH,"//table[@id='tablepress-{}']/tbody/tr/td[2]".format(i))
    for d in date:
        fund_deals['Date'].append(d.text)
        
    # scraping data of startup name
    startup_name = driver.find_elements(By.XPATH,"//table[@id='tablepress-{}']/tbody/tr/td[3]".format(i))
    for name in startup_name:
        fund_deals['Startup Name'].append(name.text)
      
    #scraping data of industry or vertical
    industry = driver.find_elements(By.XPATH,"//table[@id='tablepress-{}']/tbody/tr/td[4]".format(i))
    for ind in industry:
        fund_deals['Industry/Vertical'].append(ind.text)
        
    
    #scraping data of sub-vertical
    sub_vertical = driver.find_elements(By.XPATH,"//table[@id='tablepress-{}']/tbody/tr/td[5]".format(i))
    for sv in sub_vertical:
        fund_deals['Sub_Vertical'].append(sv.text)
        
        
    # scraping data of location
    location = driver.find_elements(By.XPATH,"//table[@id='tablepress-{}']/tbody/tr/td[6]".format(i))
    for loc in location:
        fund_deals['Location'].append(loc.text)
        
        
    # scraping data of investor
    investor = driver.find_elements(By.XPATH,"//table[@id='tablepress-{}']/tbody/tr/td[7]".format(i))
    for invest in investor:
        fund_deals['Investor'].append(invest.text)
        
        
    # scraping data of investment type
    investment_type = driver.find_elements(By.XPATH,"//table[@id='tablepress-{}']/tbody/tr/td[8]".format(i))
    for invtype in investment_type:
        fund_deals['Investment Type'].append(invtype.text)
        
        
    # scraping data of amount
    amount =driver.find_elements(By.XPATH,"//table[@id='tablepress-{}']/tbody/tr/td[9]".format(i))
    for amt in amount:
        fund_deals['Amount(in USD)'].append(amt.text)

In [59]:
# checking lengths of all scraped data
print(len(fund_deals['Date']),
len(fund_deals['Startup Name']),
len(fund_deals['Industry/Vertical']),
len(fund_deals['Sub_Vertical']),
len(fund_deals['Location']),
len(fund_deals['Investor']),
len(fund_deals['Investment Type']),
len(fund_deals['Amount(in USD)'] 
))

30 30 30 30 30 30 30 30


In [60]:
# creating DataFrame for scraped data
fund_data = pd.DataFrame(fund_deals)
fund_data

,Date,Startup Name,Industry/Vertical,Sub_Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [61]:
# saving data in csv file
fund_data.to_csv("trak_in.csv")

In [62]:
driver.close()

# Q7 : Write a program to scrap all the available details of best gaming laptops from digit.in.

In [63]:
# connecting to the webdriver
driver=webdriver.Chrome(r"C:\Chrome driver\Chromedriver.exe")


In [64]:
driver.get("https://www.digit.in/")

In [65]:
# searching for best Laptop
best_gam_laptops = driver.find_element(By.XPATH,"//div[@class='listing_container']//ul//li[9]").click()
time.sleep(3)

In [66]:
# creating empty list
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Weight = []
Dimensions = []
Graph_proc = []
Price = []

In [67]:
#scraping the data of laptop names
try:
    laptop_name = driver.find_elements(By.XPATH,'//div[@class="TopNumbeHeading"]')
    for name in laptop_name:
        Laptop_Name.append(name.text)
except NoSuchElementException:
    pass

#scraping the data of operating system
try:
    op_sys = driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[1]/div/div")
    for os in op_sys:
        Operating_sys.append(os.text)
except NoSuchElementException:
    pass


#scraping data of display of the Laptop
try:
    display = driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[2]/div/div")
    for disp in display:
        Display.append(disp.text)
except NoSuchElementException:
    pass


# scraping data of processor
try:
    processor = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[5]/td[3]")
    for pro in processor:
        Processor.append(pro.text)
except NoSuchElementException:
    pass


# scraping the data of memory
try:
    memory = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")
    for memo in memory:
        Memory.append(memo.text)
except NoSuchElementException:
    pass


# scraping data of weight
try:
    weight = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]")
    for wgt in weight:
        Weight.append(wgt.text)
except NoSuchElementException:
    pass


# scraping data of dimensions
try:
    dimension = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]")
    for dim in dimension:
        Dimensions.append(dim.text)
except NoSuchElementException:
    pass


# scraping data of graph processor
try:
    graph = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")
    for gra in graph:
        Graph_proc.append(gra.text)
except NoSuchElementException:
    pass


# scraping the data of price
try:
    price = driver.find_elements(By.XPATH,"//td[@class='smprice']")
    for pri in price:
        Price.append(pri.text.replace('₹ ','Rs'))
except NoSuchElementException:
    pass

In [68]:
print(len(Laptop_Name),
len(Operating_sys),
len(Display),
len(Processor),
len(Memory),
len(Weight),
len(Dimensions),
len(Graph_proc),
len(Price))

10 10 10 10 10 10 10 10 10


In [69]:
#creating DataFrame for scraped data
Gaming_Laptop=pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = Laptop_Name
Gaming_Laptop['Operating System'] =Operating_sys
Gaming_Laptop['Display'] = Display
Gaming_Laptop['Processor'] = Processor
Gaming_Laptop['Memory'] = Memory
Gaming_Laptop['Weight'] = Weight
Gaming_Laptop['Dimensions'] = Dimensions
Gaming_Laptop['Graphical Processor'] = Graph_proc
Gaming_Laptop['Price'] = Price
Gaming_Laptop

,Laptop Name,Operating System,Display,Processor,Memory,Weight,Dimensions,Graphical Processor,Price
0,1.\nMSI Titan GT77-12UHS,Windows 11 Home OS,"17.3"" (3840 x 2160) Display",Windows 11 Home,64 GB DDR5 RAM & 2 TB SSD,397 x 330 x 23 mm dimension & 3.3 kg weight,397 x 330 x 23 mm dimension & 3.3 kg weight,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,N/A
1,2.\nAlienware X17 R2,Windows 11 Home OS,"17.3"" (1920 x 1080) Display",Windows 11 Home,32 GB DDR5 RAM & 1 TB SSD,299 x 359 x 21 mm dimension & 2.96 kg weight,299 x 359 x 21 mm dimension & 2.96 kg weight,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...,"Rs389,990"
2,3.\nAcer Predator Triton 500 SE PT516-52s,Windows 11 Home OS,"16"" (2560 x 1600) Display",Windows 11 Home,32 GB DDR5 RAM & 2 TB SSD,358 x 262 x 19.9 mm dimension & 2.4 kg weight,358 x 262 x 19.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTXTM 3070 Ti Graphic...,"Rs300,000"
3,4.\nOmen By HP (16-B1371TX),Windows 11 Home OS,"16.1"" (2560 x 1440) Display",Windows 11 Home,8 GB DDR5 RAM & 1 TB SSD,369 x 248 x 23 mm dimension & 2.32 kg weight,369 x 248 x 23 mm dimension & 2.32 kg weight,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,"Rs176,290"
4,5.\nAcer Predator Helios 300 AN515-45 (NH.QBRSI.0,Windows 11 Home OS,"15.6"" (2560 x 1440) Display",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,363 x 255 x 23.9 mm dimension & 2.4 kg weight,363 x 255 x 23.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,"Rs172,999"
5,6.\nMSI Delta 15 (A5EFK-083IN),Windows 11 Home OS,"15.6"" (1920 x 1080) Display",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,357 x 247 x 19 mm dimension & 1.9 kg weight,357 x 247 x 19 mm dimension & 1.9 kg weight,10 GB DDR6 AMD Radeon RX 6700M Graphics card,"Rs188,990"
6,7.\nOmen By HP (16-C0141AX),Windows 11 Home OS,"16.1"" (2560 x 1440) Display",Windows 11 Home,16 GB DDR4 RAM & 1 TB NVMe,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,"Rs124,990"
7,8.\nLenovo Legion 5i Pro (82RF00MGIN),Windows 11 Home OS,"16"" (2560 x 1600) Display",Windows 11 Home,16 GB DDR5 RAM & 1 TB SSD,360 x 264 x 20 mm dimension & 2.49 kg weight,360 x 264 x 20 mm dimension & 2.49 kg weight,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,"Rs168,500"
8,9.\nAlienware M15 R5 Ryzen Edition Icc-C780001win,Windows 11 Home OS,"15.6"" (1920 x 1080) Display",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,"Rs144,990"
9,10.\nLenovo Slim 7 Gen 6 (82K8002JIN),Windows 11 Home OS,15.6 MP | NA Display,Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,356 x 252 x 16 mm dimension & 1.9 kg weight,356 x 252 x 16 mm dimension & 1.9 kg weight,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,"Rs131,990"


In [70]:
# saving the data to csv
Gaming_Laptop.to_csv("Gaming_Laptops.csv")

In [71]:
driver.close()

# Q8 : Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry

In [73]:
# connecting to the webdriver
driver=webdriver.Chrome(r"C:\Chrome driver\Chromedriver.exe")


In [75]:
driver.get("https://www.forbes.com/?sh=4cce54162254")

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: unable to connect to renderer)
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00CB1ED3+2236115]
	Ordinal0 [0x00C492F1+1807089]
	Ordinal0 [0x00B566FD+812797]
	Ordinal0 [0x00B48DEF+757231]
	Ordinal0 [0x00B5C94F+837967]
	Ordinal0 [0x00BB590A+1202442]
	Ordinal0 [0x00BA18A6+1120422]
	Ordinal0 [0x00B7A73D+960317]
	Ordinal0 [0x00B7B71F+964383]
	GetHandleVerifier [0x00F5E7E2+2743074]
	GetHandleVerifier [0x00F508D4+2685972]
	GetHandleVerifier [0x00D42BAA+532202]
	GetHandleVerifier [0x00D41990+527568]
	Ordinal0 [0x00C5080C+1837068]
	Ordinal0 [0x00C54CD8+1854680]
	Ordinal0 [0x00C54DC5+1854917]
	Ordinal0 [0x00C5ED64+1895780]
	BaseThreadInitThunk [0x767AFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77697BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77697B8E+238]


In [80]:
# scraping required data from the web page
# creating empty lists
Rank = []
Person_Name = []
Net_worth = []
Age = []
Citizenship = []
Source = []
Industry = []


while(True):
    
    # scraping the data of rank of the billionaires
    rank_tag = driver.find_elements(By.XPATH,"//div[@class='rank']")
    for rank in rank_tag:
        Rank.append(rank.text)
    time.sleep(1)
    
    
 
    # scraping the data  of names of the billionaires
    name_tag = driver.find_elements(By.XPATH,"//div[@class='personName']/div")
    for name in name_tag:
        Person_Name.append(name.text)
    time.sleep(1)
    
    
    # scraping the data of age of the billionaires
    age_tag = driver.find_elements(By.XPATH,"//div[@class='age']/div")
    for age in age_tag:
        Age.append(age.text)
    time.sleep(1)
    
    
    # scraping the data of citizenship of the billionaires
    cit_tag = driver.find_elements(By.XPATH,"//div[@class='countryOfCitizenship']")
    for cit in cit_tag:
        Citizenship.append(cit.text)
    time.sleep(1)
    
    
    # scraping the data of source of income of the billionaires
    sour_tag = driver.find_elements(By.XPATH,"//div[@class='source']")
    for sour in sour_tag:
        Source.append(sour.text)
    time.sleep(1)
    
    
    # scraping data of industry of the billionaires
    ind_tag = driver.find_elements(By.XPATH,"//div[@class='category']//div")
    for ind in ind_tag:
        Industry.append(ind.text)
    time.sleep(1)
    
    
    # scraping data of net_worth of billionaires
    net_tag = driver.find_elements(By.XPATH,"//div[@class='netWorth']/div")
    for net in net_tag:
        Net_worth.append(net.text)
    time.sleep(1)
    
    
    # clicking on next button
    try:
        next_button = driver.find_elements(By.XPATH,"//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00CB1ED3+2236115]
	Ordinal0 [0x00C492F1+1807089]
	Ordinal0 [0x00B566FD+812797]
	Ordinal0 [0x00B49449+758857]
	Ordinal0 [0x00B492C1+758465]
	Ordinal0 [0x00B48113+753939]
	Ordinal0 [0x00B48748+755528]
	Ordinal0 [0x00B515D2+792018]
	Ordinal0 [0x00B5CB15+838421]
	Ordinal0 [0x00B5FEB0+851632]
	Ordinal0 [0x00B48ADC+756444]
	Ordinal0 [0x00B5C94F+837967]
	Ordinal0 [0x00BB590A+1202442]
	Ordinal0 [0x00BA18A6+1120422]
	Ordinal0 [0x00B7A73D+960317]
	Ordinal0 [0x00B7B71F+964383]
	GetHandleVerifier [0x00F5E7E2+2743074]
	GetHandleVerifier [0x00F508D4+2685972]
	GetHandleVerifier [0x00D42BAA+532202]
	GetHandleVerifier [0x00D41990+527568]
	Ordinal0 [0x00C5080C+1837068]
	Ordinal0 [0x00C54CD8+1854680]
	Ordinal0 [0x00C54DC5+1854917]
	Ordinal0 [0x00C5ED64+1895780]
	BaseThreadInitThunk [0x767AFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77697BBE+

In [ ]:
print(len(Rank),
len(Person_Name),
len(Net_worth),
len(Age),
len(Citizenship),
len(Source),
len(Industry))

In [ ]:
# framing Data
Billionaires = pd.DataFrame({})
Billionaires['Rank'] = Rank
Billionaires['Name'] = Person_Name
Billionaires['Net Worth'] = Net_worth
Billionaires['Age'] = Age
Billionaires['Citizenship'] = Citizenship
Billionaires['Source'] = Source
Billionaires['Industry'] = Industry
Billionaires

In [ ]:
# saving dataset in csv
Billionaires.to_csv('Forbes_Billionaires.csv')

In [ ]:
driver.close()

# Q9 : Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [85]:
# connecting to the webdriver
driver=webdriver.Chrome(r"C:\Chrome driver\Chromedriver.exe")

In [86]:
driver.get("https://www.youtube.com/")

In [97]:
# finding element for search bar
search_bar = driver.find_element(By.XPATH,"//div[@class='ytd-searchbox-spt']/input")
search_bar.send_keys("GOT")      # entering video name
time.sleep(2)

In [98]:
#clicking on search button
search_btn = driver.find_element(By.XPATH,"/html/body/ytd-app/div[1]/div/ytd-masthead/div[3]/div[2]/ytd-searchbox/button/yt-icon")
search_btn.click()
time.sleep(2)

In [99]:
# clicking on first video
video = driver.find_element(By.XPATH,"//yt-formatted-string[@class='style-scope ytd-video-renderer']")
video.click()

In [101]:
# 1000 times we scroll down by 10000 in order to generate more comments
for _ in range(3000):
    driver.execute_script("window.scrollBy(0,10000)")

In [102]:
# creating empty lists
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

In [103]:
# scraping  hostel name
try:
    cm = driver.find_elements(By.XPATH,'//yt-formatted-string[@id="content-text"]')
    for i in cm:
        comments.append(i.text)
except NoSuchElementException:
    comments.append('-')

In [93]:
# scrape time when comment was posted
tm = driver.find_elements(By.XPATH,'//yt-formatted-string[@class="published-time-text style-scope ytd-comment-renderer"]')
for i in tm:
    Time.append(i.text)
    
for i in range(0,len(Time),2):
    comment_time.append(Time[i])
time.sleep(4)

In [104]:
# scrape the comment likes
like = driver.find_elements(By.XPATH,"//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for i in like:
    Likes.append(i.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])

In [105]:
print(len(comments),len(comment_time),len(No_of_Likes))

180 0 180


In [109]:
# creating dataframe for scraped data

Youtube = pd.DataFrame({})
Youtube['Comment'] = comments[:500]
Youtube['Comment Time'] = comment_time[:500]
Youtube['Comment Upvotes'] = No_of_Likes[:500]
Youtube


ValueError: Length of values (0) does not match length of index (180)

In [ ]:
#saving the dataframe to csv
Youtube.to_csv("Youtube GOT Comments.csv")

In [ ]:
driver.close()

# Q10 : Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [107]:
# connecting to the webdriver
driver=webdriver.Chrome(r"C:\Chrome driver\Chromedriver.exe")

In [108]:
driver.get("https://www.hostelworld.com/")


In [112]:
# locating the location search bar
search_bar = driver.find_element(By.XPATH,'//input[@id="search-input-field"]')

In [113]:
# entering London in search bar
search_bar.send_keys("London")

In [114]:
# select London
London = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div')
#clicking on button
London.click()


In [116]:
# do click on Let's Go button
search_btn = driver.find_element(By.XPATH,'//button[@class="button primary large"]')
search_btn.click()

In [117]:
# creating empty list & find required data
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description = []
url = []

In [118]:
# scraping  hostel name
try:
    name = driver.find_elements(By.XPATH,'//h2[@class="title title-6"]')
    for i in name:
        hostel_name.append(i.text)
except NoSuchElementException:
    hostel_name.append('-')

In [119]:
# scraping distance from city centre
try:
    dist = driver.find_elements(By.XPATH,'//span[@class="description"]')
    for i in dist:
        distance.append(i.text.replace('Hostel - ',''))
except NoSuchElementException:
    distance.append('-')

In [120]:
for i in driver.find_elements(By.XPATH,"//div[@class='prices-col']"):   
    # scraping privates from price
    try:
        pvt_price = driver.find_element(By.XPATH,"//a[@class='prices']//div[1]//div")
        pvt_prices.append(pvt_price.text)
    except NoSuchElementException:
        pvt_prices.append('-')

In [121]:
for i in driver.find_elements(By.XPATH,"//div[@class='prices-col']"):          
    # scraping dorms from price
    try:
        dorms = driver.find_element(By.XPATH,"//a[@class='prices']//div[2]/div")
        dorms_price.append(dorms.text)
    except NoSuchElementException:
        dorms_price.append('-')

In [128]:
# scraping facilities
try:
    fac1 = driver.find_elements(By.XPATH,"//div[@class='has-wifi']")
    fac2 = driver.find_elements(By.XPATH,"//div[@class='has-sanitation']")
    for i in fac1:
        for j in fac2:
            facilities.append(i.text +', '+ j.text)
except NoSuchElementException:
    facilities.append('-')

In [129]:
#fetching url of each hostel
p_url = driver.find_elements(By.XPATH,"//div[@class='prices-col']//a[2]")
for i in p_url:
    url.append(i.get_attribute("href"))
        

In [130]:
for i in url:
    driver.get(i)
    time.sleep(3)
    
    # scraping ratings
    try:
        rat = driver.find_element(By.XPATH,"//div[@class='score orange big' or @class='score gray big']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
        
    # scraping total review
    try:
        rws = driver.find_element(By.XPATH,"//div[@class='reviews']")
        reviews.append(rws.text.replace('Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
        
        
    # fetching over all review
    try:
        overall = driver.find_element(By.XPATH,"//div[@class='keyword']//span")
        over_all.append(overall.text)
    except NoSuchElementException:
        over_all.append('-')
        
    # fetching property description
    try:
        disc = driver.find_element(By.XPATH,"//div[@class='content']")
        description.append(disc.text)
    except NoSuchElementException:
        over_all.append('-')


In [131]:
print(len(hostel_name),
len(distance),
len(pvt_prices),
len(dorms_price),
len(rating),
len(reviews),
len(over_all),
len(facilities),
len(description),
len(url))

30 30 30 30 60 60 60 1100 60 30


In [132]:
# creating DataFrame
Hostel = pd.DataFrame({})
Hostel['Hostel Name'] = hostel_name
Hostel['Distance from City Centre'] = distance
Hostel['Ratings'] = rating
Hostel['Total Reviews'] = reviews
Hostel['Overall Reviews'] = over_all
Hostel['Privates from Price'] = pvt_prices
Hostel['Dorms from Price'] = dorms_price
Hostel['Facilities'] = facilities[:14]
Hostel['Description'] = description
Hostel

ValueError: Length of values (60) does not match length of index (30)

In [133]:
# saving the dataset to csv
Hostel.to_csv("London_Hostels.csv")

In [134]:
driver.close()